# Shadow Finder ☀️🌐

A tool to estimate the points on the Earth's surface where a shadow of a particular length could occur, for geolocation purposes.

Using an object's height, the lenth of its shadow, the date and the time, this code estimates the possible locations of that shadow.

> <font color='#ffc107'>Important:</font> The shadow length must be measured at right angles to the object 📐 This means that you might have to correct for the perspective of an image before using this tool. 

In [ ]:
# @title Find a Shadow 🔎 { display-mode: "form" }

# @markdown ### ⬅️ Click to find possible locations that match the below information (takes around 10 - 20 seconds)

# @markdown Object and shadow are measured at right angles in arbitrary units
object_height = 10  # @param {type:"number"} Height of object in arbitrary units
shadow_length = 8  # @param {type:"number"} Length of shadow in arbitrary units

# @markdown Date and time can be given in UTC or local time (set `time type` accordingly), using the time format hh:mm:ss
date = "2024-02-29"  # @param {type:"date"}
time = "12:00:00"  # @param {type:"string"}
time_type = "utc" # @param ["utc", "local"]
include_sea_locations = False # @param {type:"boolean"}
# Create output files
output = f"./shadowfinder_{object_height}_{shadow_length}_{date}T{time}.png"
logfile = f"./shadowfinder_{object_height}_{shadow_length}_{date}T{time}.log"

# Imports
![ ! -f "deps_loaded" ] && pip install shadowfinder >> {logfile} 2>&1 && touch deps_loaded

from shadowfinder.shadowfinder import ShadowFinder
import datetime

datetime_date = datetime.datetime.strptime(date, "%Y-%m-%d").date()
datetime_time = datetime.datetime.strptime(time, "%H:%M:%S").time()

date_time = datetime.datetime.combine(
    datetime_date, datetime_time, tzinfo=datetime.timezone.utc
)  # Date and time of interest

if "finder" not in locals():
    finder = ShadowFinder()
    # check if timezones.npz exists
    try:
        finder.load_timezones()
    except FileNotFoundError:
        finder.generate_lat_lon_grid()
        finder.save_timezones()

finder.set_details(object_height, shadow_length, date_time, process_sea=include_sea_locations, time_format=time_type)
finder.find_shadows()
finder.plot_shadows()